In [32]:
import cupy as cp

ModuleNotFoundError: No module named 'cupy'

In [1]:
import requests
import pandas as pd
import datetime 

In [2]:
api_key = 'test_b375f597f54e36891fce7c2f065cd1a97f9575f16b3799c740322f3ed9df9bc8efe8d04e6d233bd35cf2fabdeb93fb0d'
headers = {'x-nxopen-api-key': api_key}

FCO 매치 종류 및 Division 식별자 확인

In [3]:
match_response = requests.get('https://open.api.nexon.com/static/fconline/meta/matchtype.json', 
                         headers = headers)

match_json = match_response.json()
match_df = pd.DataFrame(match_json)
match_df = match_df.rename(columns={'matchtype' : 'matchType'}) 

divison_response = requests.get('https://open.api.nexon.com/static/fconline/meta/division.json', 
                         headers = headers)

division_json = divison_response.json()
division_df = pd.DataFrame(division_json)
division_df = division_df.rename(columns={'divisionId' : 'division'}) 

match_dict = match_df.set_index('matchType')['desc'].to_dict()
division_dict = division_df.set_index('division')['divisionName'].to_dict()

In [4]:
match_dict

{30: '리그 친선',
 40: '클래식 1on1',
 50: '공식경기',
 52: '감독모드',
 60: '공식 친선',
 204: '볼타 친선',
 214: '볼타 공식',
 224: '볼타 AI대전',
 234: '볼타 커스텀'}

In [5]:
division_dict

{800: '슈퍼챔피언스',
 900: '챔피언스',
 1000: '슈퍼챌린지',
 1100: '챌린지1',
 1200: '챌린지2',
 1300: '챌린지3',
 2000: '월드클래스1',
 2100: '월드클래스2',
 2200: '월드클래스3',
 2300: '프로1',
 2400: '프로2',
 2500: '프로3',
 2600: '세미프로1',
 2700: '세미프로2',
 2800: '세미프로3',
 2900: '유망주1',
 3000: '유망주2',
 3100: '유망주3'}

FCO 유저별 매치 기록 확인}

In [6]:
api_key = 'test_b375f597f54e36891fce7c2f065cd1a97f9575f16b3799c740322f3ed9df9bc8efe8d04e6d233bd35cf2fabdeb93fb0d'
headers = {'x-nxopen-api-key': api_key}

In [7]:
user_metadata_df = pd.DataFrame()
user_match_df = pd.DataFrame()

In [8]:
nickname = '김정현의하수인'

In [9]:
users_ouid_params = {'nickname' : nickname}


resposne_user_ouid = requests.get('https://open.api.nexon.com/fconline/v1/id?',  
                    params = users_ouid_params, headers = headers)

user_ouid = resposne_user_ouid.json()
print('ouid = ', user_ouid.get('ouid'))


ouid =  415f3d938ab945a0f69aeecbeb68d488


In [10]:
user_params = {'ouid' : '415f3d938ab945a0f69aeecbeb68d488', 'matchtype' : '40'}


response_user_match = requests.get('https://open.api.nexon.com/fconline/v1/user/match?',  
                    params = user_params, headers = headers)

new_user_info = response_user_match.json()
new_user_info_df = pd.DataFrame([new_user_info])

In [11]:
new_user_info

['673a0a53ca98089a4d566e5c',
 '673a07e06c583a338765b01d',
 '673a055f4a2132424708d9e0',
 '6739fab347aa2458b3d80839',
 '6739f829b5f708536fb366fe',
 '6739f5d82951cc61982efcf5',
 '6739f20aeb3b064b0d94ff65',
 '6739ef6ce87465c25e3f84f8',
 '6739e6fda1cd2db9f4d6bbf8',
 '6739e2cd16d3c7ce7266661b',
 '6730b98e34021b471c94bba9',
 '6730b73a125d54a82d836951',
 '6730b4eb5ca892ede1e32ebb',
 '6730b28c4b97846f96b689ab',
 '6730b0340d7c37140bffdea4',
 '6730a7b2b0c7c011a718c29a',
 '6730a54b8ddcbc0844351e92',
 '6730a2277c4ab47bfa33dd89',
 '67309fdb4484b93010af47e5',
 '67309d7c313753d963778a03',
 '6727777dbc8d657b63ab0e20',
 '672774fe706e2750fac4c823',
 '672772c95efc91d997542532',
 '67277064ef8c931e12485dac',
 '67276de797394e4b30b3b929',
 '67276baa167ce01c22d0018b',
 '672767c21761a37cc237457f',
 '67276546ed95485d7dd94700',
 '672762df0cee6b48b165ff1a',
 '6727608fa92b4c7375a867e8',
 '67275dfb4c6c227cfde719e4']

FCO 유저별 기본정보 확인

In [12]:
def get_user_metadata(ouid : str, user_metadata_df : pd.DataFrame) -> pd.DataFrame : 
    '''
    Input
        nickname (str)    : Nickname want to search in FCO gaem
        user_metadata_df (pd.DataFrame)    : user_metadata_df, To update user_metadata_df with result of searching nickname

    Output 
        user_metadata_df (pd.DataFrame)    : user_metadata_df, After updating user_metadata_df with result of searching nickname 
            * [ouid, nickname, level]
    '''

    user_params = {'ouid' : ouid}

    response_user_data = requests.get('https://open.api.nexon.com/fconline/v1//user/basic?',   # Get user_metadata
                    params = user_params, headers = headers)
    print(f"Status Code for user_data: {response_user_data.status_code}") 
    
    try : 
        new_user_info = response_user_data.json()
        
        if 'error' in new_user_info:
            return user_metadata_df
        
        else :     
            new_user_info_df = pd.DataFrame([new_user_info])

            if user_metadata_df.empty or ouid not in user_metadata_df['ouid'].values:
                user_metadata_df = pd.concat([user_metadata_df, new_user_info_df], ignore_index=True)

    except requests.exceptions.JSONDecodeError:    # If the response is not in valid JSON format. 
        print("Response is not a valid.")
        
    except requests.exceptions.RequestException as e:    # If the response is invalid responses,
        print(f"Request failed: {e}")

    return user_metadata_df

FCO 유저별 매치별 최고 등급 확인

In [13]:
def get_user_match_division_grade(ouid : str, user_match_grade_df : pd.DataFrame) -> pd.DataFrame : 
    '''
    Input
        nickname (str)    : Nickname want to search in FCO gaem
        user_match_grade_df (pd.DataFrame)    : user_match_grade_df, To update user_match_grade_df with result of searching nickname
        
    Output 
        user_match_grade_df_up (pd.DataFrame)    : user_match_grade_df, After updating user_match_grade_df with result of searching nickname 
            * [ouid, matchTyle, division, achievementDate]
    '''
    
    user_params = {'ouid' : ouid}
    
    response_user_match_grade = requests.get('https://open.api.nexon.com/fconline/v1//user/maxdivision?',     # Get user_match
                                             params = user_params, headers = headers)
    print(f"Status Code for user_match: {response_user_match_grade.status_code}") 
    
    try : 
        new_user_match_grade = response_user_match_grade.json()
        
        if isinstance(new_user_match_grade, dict) and 'error' in new_user_match_grade:
            return user_match_grade_df
        
        if not new_user_match_grade:
            print("Response JSON is empty")
            return user_match_grade_df

        new_user_match_grade_df = pd.DataFrame(new_user_match_grade)

        new_user_match_grade_df['ouid'] = ouid
        new_user_match_grade_df['division'] = new_user_match_grade_df['division'].astype(int).map(division_dict)
        new_user_match_grade_df['matchType'] = new_user_match_grade_df['matchType'].astype(int).map(match_dict) 
        new_user_match_grade_df['achievementDate'] = pd.to_datetime(new_user_match_grade_df['achievementDate'])
        
        if user_match_grade_df.empty or ouid not in user_match_grade_df['ouid'].values : 
            user_match_grade_df_up = pd.concat([user_match_grade_df, new_user_match_grade_df], ignore_index=True)

        else:
            user_match_grade_df = user_match_grade_df[user_match_grade_df['ouid'] != ouid]
            user_match_grade_df_up = pd.concat([user_match_grade_df, new_user_match_grade_df], ignore_index=True)
            
    except requests.exceptions.JSONDecodeError:    # If the response is not in valid JSON format. 
        print("Response is not a valid.")
        
    except requests.exceptions.RequestException as e:    # If the response is invalid responses,
        print(f"Request failed: {e}")
        
    return user_match_grade_df_up

FCO 유저 닉네임 통한 DF 최신화

In [14]:
def get_user_metadata_match(nickname : str, 
                            user_metadata_df : pd.DataFrame, 
                            user_match_df : pd.DataFrame) -> pd.DataFrame : 
    '''
    Input
        nickname (str)    : Nickname want to search in FCO gaem
        user_metadata_df (pd.DataFrame)    : User_metadata_df, To update user_metadata with result of searching nickname
        user_match_df (pd.DataFrame)    : User_metadata_df, To update user_match_df with result of searching nickname

        
    Output 
        user_metadata_df_up (pd.DataFrame)    : user_metadata_df, After updating user_metadata with result of searching nickname 
            * [ouid, nickname, level]
        user_match_df_up (pd.DataFrame)    : user_match_df, After updating user_match_df with result of searching nickname 
            * [ouid, matchTyle, division, achievementDate]
    '''
    
    user_params = {'nickname' : nickname}

    response_ouid = requests.get('https://open.api.nexon.com/fconline/v1/id?',   # Get user_ouid
                            params = user_params, headers = headers)
    print(f"Status Code for user_id: {response_ouid.status_code}") 

    try:
        new_user_json = response_ouid.json()
        new_user_ouid = new_user_json.get('ouid')
        
        user_metadata_df_up = get_user_metadata(new_user_ouid, user_metadata_df)
        user_match_df_up = get_user_match_division_grade(new_user_ouid, user_match_df)
        
    except requests.exceptions.JSONDecodeError:    # If the response is not in valid JSON format. 
        print("Response is not a valid.")
        
    except requests.exceptions.RequestException as e:    # If the response is invalid responses,
        print(f"Request failed: {e}")
        
    return user_metadata_df_up, user_match_df_up

In [15]:
user_metadata_df, user_match_df = get_user_metadata_match('김정현의하수인', user_metadata_df, user_match_df)

Status Code for user_id: 200
Status Code for user_data: 200
Status Code for user_match: 200


In [16]:
user_metadata_df

,ouid,nickname,level
0,415f3d938ab945a0f69aeecbeb68d488,김정현의하수인,428


In [17]:
user_match_df

,matchType,division,achievementDate,ouid
0,공식경기,월드클래스3,2023-03-01 01:23:17,415f3d938ab945a0f69aeecbeb68d488
1,감독모드,챌린지3,2024-03-27 13:49:26,415f3d938ab945a0f69aeecbeb68d488
2,볼타 공식,세미프로1,2020-08-16 09:59:37,415f3d938ab945a0f69aeecbeb68d488


In [1]:
# goalTime 변환을 위해 주어진 공식을 적용한 변환 로직을 작성합니다.
goal_time = 16779725
half_time_interval = 2**24

if 0 <= goal_time < half_time_interval:
    # 전반전
    converted_time = goal_time
elif half_time_interval <= goal_time < 2 * half_time_interval:
    # 후반전
    converted_time = goal_time - half_time_interval + 45 * 60
elif 2 * half_time_interval <= goal_time < 3 * half_time_interval:
    # 연장 전반전
    converted_time = goal_time - 2 * half_time_interval + 90 * 60
elif 3 * half_time_interval <= goal_time < 4 * half_time_interval:
    # 연장 후반전
    converted_time = goal_time - 3 * half_time_interval + 105 * 60
elif 4 * half_time_interval <= goal_time < 5 * half_time_interval:
    # 승부차기
    converted_time = goal_time - 4 * half_time_interval + 120 * 60

print(converted_time//60, converted_time%60)

86 49


In [2]:
match_user = {'ouid': 'c57dbb7c393f3cb5f26160aaf8b8a187', 'nickname': 'trashJM', 'matchDetail': {'seasonId': 202412, 'matchResult': '패', 'matchEndType': 2, 'systemPause': 1, 'foul': 0, 'injury': 0, 'redCards': 0, 'yellowCards': 0, 'dribble': 0, 'cornerKick': 0, 'possession': 50, 'offsideCount': 0, 'averageRating': 3.66667, 'controller': 'keyboard'}, 'shoot': {'shootTotal': 0, 'effectiveShootTotal': 0, 'shootOutScore': 0, 'goalTotal': 0, 'goalTotalDisplay': 0, 'ownGoal': 0, 'shootHeading': 0, 'goalHeading': 0, 'shootFreekick': 0, 'goalFreekick': 0, 'shootInPenalty': 0, 'goalInPenalty': 0, 'shootOutPenalty': 0, 'goalOutPenalty': 0, 'shootPenaltyKick': 0, 'goalPenaltyKick': 0}, 'shootDetail': [], 'pass': {'passTry': 1, 'passSuccess': 0, 'shortPassTry': 0, 'shortPassSuccess': 0, 'longPassTry': 0, 'longPassSuccess': 0, 'bouncingLobPassTry': 0, 'bouncingLobPassSuccess': 0, 'drivenGroundPassTry': 0, 'drivenGroundPassSuccess': 0, 'throughPassTry': 0, 'throughPassSuccess': 0, 'lobbedThroughPassTry': 0, 'lobbedThroughPassSuccess': 0}, 'defence': {'blockTry': 0, 'blockSuccess': 0, 'tackleTry': 0, 'tackleSuccess': 0}, 'player': [{'spId': 291220697, 'spPosition': 18, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 291205988, 'spPosition': 7, 'spGrade': 6, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 289013743, 'spPosition': 15, 'spGrade': 6, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 291231281, 'spPosition': 3, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 300212218, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}, {'spId': 300264645, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}, {'spId': 300237692, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}, {'spId': 300231866, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}, {'spId': 825203574, 'spPosition': 6, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 289005471, 'spPosition': 13, 'spGrade': 6, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 802007289, 'spPosition': 4, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 300218667, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}, {'spId': 825206517, 'spPosition': 19, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 283246669, 'spPosition': 17, 'spGrade': 7, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 503233934, 'spPosition': 0, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 300224081, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}, {'spId': 802202126, 'spPosition': 25, 'spGrade': 5, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 1, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 6.0}}, {'spId': 300188377, 'spPosition': 28, 'spGrade': 0, 'status': {'shoot': 0, 'effectiveShoot': 0, 'assist': 0, 'goal': 0, 'dribble': 0, 'intercept': 0, 'defending': 0, 'passTry': 0, 'passSuccess': 0, 'dribbleTry': 0, 'dribbleSuccess': 0, 'ballPossesionTry': 0, 'ballPossesionSuccess': 0, 'aerialTry': 0, 'aerialSuccess': 0, 'blockTry': 0, 'block': 0, 'tackleTry': 0, 'tackle': 0, 'yellowCards': 0, 'redCards': 0, 'spRating': 0.0}}]}

In [3]:
def position_rating(match_user_) :  
    attack_position = [i for i in range(9, 20)]
    middle_position = [i for i in range(20, 28)]
    defense_position = [i for i in range(1, 9)]
    goalkeeper_position = [0]
    
    attack_rating = []
    middle_rating = []
    defense_rating = []
    goalkeeper_rating = []
    
    for player in match_user_['player'] : 
        if player['spPosition'] == 28 :                             # 후보선수 제외
            continue

        elif player['spPosition'] in attack_position :              # 공격 평점
            attack_rating.append(player['status']['spRating'])

        elif player['spPosition'] in middle_position :              # 미들 평점
            middle_rating.append(player['status']['spRating'])

        elif player['spPosition'] in defense_position :             # 수비 평점
            defense_rating.append(player['status']['spRating'])

        else :                                                      # 골키퍼 평점 
            goalkeeper_rating.append(player['status']['spRating'])

    return round(np.mean(attack_rating), 2), round(np.mean(middle_rating), 2), round(np.mean(defense_rating), 2), round(np.mean(goalkeeper_rating), 2)